# 🔌CONEXÃO

Estabelece conexão com fontes de dados

In [1]:
import psycopg2
from psycopg2.extras import DictCursor
import fdb
from dotenv import load_dotenv
import os

load_dotenv()

# Conexões
cnx_dest = fdb.connect(
    user=os.getenv("FDB_USER"),
    password=os.getenv("FDB_PASS"),
    host=os.getenv("FDB_HOST"),
    port=int(os.getenv("FDB_PORT")),
    database=os.getenv("FDB_PATH"),
    charset="WIN1252"
)
cur_dest = cnx_dest.cursor()


cnx_orig = psycopg2.connect(
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASS"),
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DB"),
    options="-c search_path={}".format(os.getenv("PG_SCHEMA"))
)
cnx_orig.autocommit = True
cur_orig = cnx_orig.cursor(cursor_factory=DictCursor)

def commit():
    cnx_dest.commit()

# 🧰 FERRAMENTAS

In [2]:
cadest, ccustos = {}, {}
empresa = cur_dest.execute("SELECT empresa FROM cadcli").fetchone()[0]
exercicio = str(cur_dest.execute("SELECT MEXER FROM CADCLI").fetchone()[0])

def limpa_tabela(tabelas):
    for tabela in tabelas:
        cur_dest.execute(f"DELETE FROM {tabela}")
    commit()

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
        commit()
    except:
        pass

dict_modalidades = {
    1:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    2:{"Licit": "MAT / SERV - CONVITE", 
          "Modlic": "CC02",
          "Codmod": 7},
    3:{"Licit": "MAT / SERV - TOMADA", 
          "Modlic": "TOM3",
          "Codmod": 3},
    4:{"Licit": "CONCORRÊNCIA ELETRÔNICA", 
          "Modlic": "CE01",
          "Codmod": 13},
    5:{"Licit": "LEILÃO", 
          "Modlic": "LEIL",
          "Codmod": 6},
    6:{"Licit": "CONCURSO", 
          "Modlic": "CS01",
          "Codmod": 5},
    7:{"Licit": "PREGÃO PRESENCIAL", 
          "Modlic": "PP01",
          "Codmod": 8},
    8:{"Licit": "INEXIGIBILIDADE", 
          "Modlic": "IN01",
          "Codmod": 5},
    9:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    10:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    11:{"Licit": "PREGÃO ELETRÔNICO", 
          "Modlic": "PE01",
          "Codmod": 9},
    12:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    13:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    14:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    15:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    16:{"Licit": "MAT / SERV - CONVITE", 
        "Modlic": "CC02",
        "Codmod": 7},
    17:{"Licit": "MAT / SERV - CONVITE", 
        "Modlic": "CC02",
        "Codmod": 7},
    18:{"Licit": "MAT / SERV - TOMADA", 
          "Modlic": "TOM3",
          "Codmod": 3},
    19:{"Licit": "MAT / SERV - TOMADA", 
          "Modlic": "TOM3",
          "Codmod": 3},
    20:{"Licit": "CONCORRÊNCIA ELETRÔNICA", 
          "Modlic": "CE01",
          "Codmod": 13},
    21:{"Licit": "CONCORRÊNCIA ELETRÔNICA", 
          "Modlic": "CE01",
          "Codmod": 13},
    22:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    23:{"Licit": "PREGÃO ELETRÔNICO", 
          "Modlic": "PE01",
          "Codmod": 9},
    24:{"Licit": "PREGÃO ELETRÔNICO", 
          "Modlic": "PE01",
          "Codmod": 9},
    25:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    26:{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
}

def to_cp1252_safe(value, replace_with=''):
    """
    Converte uma string para cp1252 de forma segura.
    Remove ou substitui caracteres inválidos.

    :param value: valor a converter
    :param replace_with: string usada no lugar de caracteres inválidos (default = '')
    :return: valor limpo
    """
    if isinstance(value, str):
        try:
            # Tenta converter diretamente
            value.encode('cp1252')
            return value
        except UnicodeEncodeError:
            # Remove/substitui caracteres inválidos
            return value.encode('cp1252', errors='replace').decode('cp1252').replace('?', replace_with)
    return value

# Carrega dados da tabela CADEST
cur_dest.execute("SELECT codreduz, max(cadpro) FROM cadest group by 1")
if cur_dest.description is None:
    print("CADEST VAZIA!")
else:
    cadest = {k:v for k,v in cur_dest.execute("SELECT codreduz, max(cadpro) FROM cadest group by 1").fetchall()}

# Carrega dados da tabela DESFOR
fornecedores = {k:v for k,v in cur_dest.execute("SELECT codif, replace(replace(insmf,'.',''),'-','') as insmf FROM desfor").fetchall()}

# COMPRAS 

In [3]:
cur_dest.execute("""
execute block as
		begin
		DELETE FROM ICADREQ;
		DELETE FROM REQUI;
		DELETE FROM ICADPED;
		DELETE FROM CADPED;
		DELETE FROM regpreco;
		DELETE FROM regprecohis;
		DELETE FROM regprecodoc;
		DELETE FROM CADPRO_SALDO_ANT;
		DELETE FROM CADPROLIC_DETALHE_FIC;
		DELETE FROM CADPRO;
		DELETE FROM CADPRO_FINAL;
		DELETE FROM CADPRO_LANCE;
		DELETE FROM CADPRO_PROPOSTA;
		DELETE FROM PROLICS;
		DELETE FROM PROLIC;
		DELETE FROM CADPRO_STATUS;
		DELETE FROM CADLIC_SESSAO;
		DELETE FROM CADPROLIC_DETALHE;
		DELETE FROM CADPROLIC;
		DELETE FROM CADLIC;
		DELETE FROM VCADORC;
		DELETE FROM FCADORC;
		DELETE FROM ICADORC;
		DELETE FROM CADORC;
		DELETE FROM CADEST;
		DELETE FROM CENTROCUSTO;
		DELETE FROM DESTINO;
		DELETE FROM DESFORCRC_PADRAO;
		end;
""")
commit()

## CADASTRO BASE

### UNIDADES DE MEDIDA

In [4]:
limpa_tabela(("cadunimedida",))

insert = cur_dest.prep("insert into cadunimedida(sigla,descricao) values(?,?)")

cur_orig.execute("""select coalesce(substr("DESCRICAO"::text, 1, 5), 'UNID') sigla, max("DESCRICAO") descricao from "SCH"."MEDIDAS" m group by 1""")

for row in cur_orig.fetchall():
    cur_dest.execute(insert, (to_cp1252_safe(row['sigla']), to_cp1252_safe(row['descricao'])))
commit()

### GRUPO & SUBGRUPO

In [5]:
limpa_tabela(("cadsubgr", "cadgrupo"))

cur_orig.execute("""
select
	to_char("CD_SUBGRUPO", 'fm000') GRUPO,
	to_char("CD_CLASSEPR", 'fm000') SUBGRUPO,
	substr(c."DS_CLASSEPR", 1, 45) descricao,
	'N' ocultar
from
	"SCH"."CLASSEPR" c
union all
select
	to_char(g."CD_SUBGRUPO", 'fm000') GRUPO,
	'000',
	substr(g."DS_SUBGRUPO", 1, 45) descricao,
	'N' ocultar
from
	"SCH"."SUBGRUP" g
order by 2, 1
""")

for row in cur_orig:
    if row['subgrupo'] == '000':
        cur_dest.execute(f"insert into cadgrupo(grupo, nome) values('{row['grupo']}', '{row['descricao']}')")
        commit()
    else:
        cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values(?, ?, ?)", (row['grupo'], row['subgrupo'], row['descricao']))
cur_dest.execute("insert into cadgrupo(grupo, nome) values ('000', 'CONVERSAO')")
cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('000', '000', 'CONVERSAO')")
commit()

### CADEST

In [6]:
limpa_tabela(("cadest",))
cria_coluna("cadest", 'subgrupo_pai')

insert = cur_dest.prep("insert into cadest(grupo, subgrupo, codigo, cadpro, codreduz, disc1, ocultar, unid1, tipopro, usopro, subgrupo_pai) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)")

cur_orig.execute("""
with CTE as (
select
	coalesce(nullif(to_char("CD_SUBGRUPO", 'fm000'), ''), '000') GRUPO,
	case when nullif(to_char("CD_SUBGRUPO", 'fm000'), '') is null then '000' else to_char("CD_CLASSEPR", 'fm000') end SUBGRUPO,
	a."CD_PRODUTO"  codreduz,
	a."DS_PRODUTO" disc1,
	case when a."INATIVO" = '1' then 'S' else 'N' end ocultar,
	coalesce(substr(c."DESCRICAO"::text,1,4), 'UNID') unid1,
	case 
		when coalesce(nullif(trim(a."DS_TIPO_PROD"),''),'0')::int in (1,5,0) then 'P|C'		
		when a."DS_TIPO_PROD"::int = 2 then 'S|'
		when a."DS_TIPO_PROD"::int in (3, 4, 6, 7) then 'P|P'
	end tipo_uso ,
	a."QT_EST_MINIMO" quanmin,
	a."QT_EST_MAXIMO" quanmax
from
	"SCH"."PRODUTOS" a
left join "SCH"."CLASSEPR" b using ("CD_CLASSEPR")
left join "SCH"."MEDIDAS" c on CASE 
    WHEN trim(COALESCE(a."DS_UNIDADE", '')) ~ '[A-Za-z]' 
         OR trim(a."DS_UNIDADE") = '' 
    THEN 2
    ELSE a."DS_UNIDADE"::integer
END = c."CODIGO"
order by 1, 2)
select *, concat(grupo, '.', subgrupo) key from CTE
""")

cur_dest.execute("select grupo||'.'||subgrupo, cast(subgrupo as integer) subgrupo_int from cadsubgr")
subgrupos_filhos = {k: [v, 0] for k, v in cur_dest.fetchall()}

cur_dest.execute("select grupo, max(subgrupo) FROM cadsubgr GROUP BY 1")
ultimo_subgrupo_por_grupo = {k: v for k, v in cur_dest.fetchall()}

for row in cur_orig:
	subgrupo_int, codigo = subgrupos_filhos[row['key']]
	codigo += 1

	if codigo > 999:
		subgrupo_int = int(ultimo_subgrupo_por_grupo[row['grupo']]) + 1
		codigo = 1
		subgrupos_filhos[row['key']] = [subgrupo_int, codigo]
		ultimo_subgrupo_por_grupo[row['grupo']] = f'{subgrupo_int:03}'
	else:
		subgrupos_filhos[row['key']][1] = codigo

	subgrupo = f'{subgrupo_int:03}'
	codigo = f'{codigo:03}'
	cadpro = f'{row['grupo']}.{subgrupo}.{codigo}'
	tipopro, usopro = row['tipo_uso'].split('|')

	cur_dest.execute(insert, (row['grupo'], subgrupo, codigo, cadpro, row['codreduz'], to_cp1252_safe(row['disc1']), row['ocultar'], to_cp1252_safe(row['unid1']), tipopro, usopro, row['key']))
commit()

cur_dest.execute("""
INSERT INTO CADSUBGR (grupo, subgrupo, nome)
WITH CTE AS (
SELECT DISTINCT a.grupo, a.subgrupo, SUBGRUPO_PAI KEY FROM cadest a
WHERE NOT EXISTS (SELECT 1 FROM cadsubgr x WHERE a.grupo = x.grupo AND a.SUBGRUPO = x.SUBGRUPO))
SELECT a.GRUPO, a.SUBGRUPO, b.NOME FROM cte a
JOIN CADSUBGR b ON b.grupo||'.'||b.SUBGRUPO = a."KEY" 
""")
commit()

cur_dest.execute("UPDATE CADEST a SET fK_CADSUBGR = (SELECT pk_CADSUBGR FROM CADSUBGR x WHERE x.GRUPO  = a.Grupo AND x.SUBGRUPO = a.subgrupo)")
commit()

cadest = {k: v for k,v in cur_dest.execute("select codreduz, max(cadpro) from cadest group by codreduz").fetchall()}

### ALMOXARIFADOS

In [7]:
limpa_tabela(("destino",))

insert = cur_dest.prep("""insert into destino(cod,desti,empresa) values(?,?,?)""")

cur_orig.execute("""
select
	to_char("CD_LOCAL",'fm000000000') cod,
	l."DS_LOCAL" desti
from
	"SCH"."LOCALMOX" l
""")

for row in cur_orig.fetchall():
    cur_dest.execute(insert, (row['cod'], row['desti'], empresa))
commit()

### CENTROS DE CUSTO

In [8]:
limpa_tabela(("centrocusto",))
cria_coluna("centrocusto", "codant")

try:
    cur_dest.execute(f"insert into destino(cod,desti,empresa) values('000000000','CONVERSAO',{empresa})")
except fdb.DatabaseError as e:
    print(f"Erro ao inserir destino: {e}")
else:
    commit()

insert = cur_dest.prep("""
    insert into centrocusto (codccusto, descr, ccusto, ocultar, empresa, poder, orgao, destino, codant) values (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select
	"RECNUM" CODIGO,
	substr("DESCRICAO", 1, 60) as descr,
	"ID_CADSET" CODANT
from
	"SCH"."CADSET"
""")

for row in cur_orig:
    cur_dest.execute(insert, (row['codigo'], row['descr'], '001', 'N', empresa, '02', '01', '000000000', row['codant']))
    ccustos[row['codant']] = row['codigo']
cur_dest.execute(insert, (0, 'CONVERSAO', '001', 'N', empresa, '02', '01', '000000000', 0))
commit()

## COTAÇÕES

### CADORC

In [9]:
limpa_tabela(('cadorc',))

insert = cur_dest.prep("""
insert into cadorc
	(id_cadorc,
	num,
	ano,
	numorc,
	dtorc,
	descr,
	prioridade,
	obs,
	status,
	liberado,
	codccusto,
	liberado_tela,
	empresa) values
	(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select
	"NR_COTACAO" ID_CADORC,
	TO_CHAR("NR_COTACAO", 'fm00000') num,
	extract(year from "DT_COTACAO") ano,
	concat(TO_CHAR("NR_COTACAO", 'fm00000/'), extract(year from "DT_COTACAO")::int%2000) numorc,
	"DT_COTACAO" dtorc,
	substr("OBSERVACAO",1, 1024) descr,
	'NORMAL' prioridade,
	"OBSERVACAO" obs,
	'LC' status,
	'S' liberado,
	'L' liberado_tela,
	trim("ID_CADSET") codccusto
from
	"SCH"."COTACOES"
order by "NR_COTACAO","DT_COTACAO"
""")

for row in cur_orig:
    try:
        row['codccusto'] = ccustos[row['codccusto']]
    except:
        print(f"Erro ao mapear codccusto: {row['codccusto']}")
        row['codccusto'] = 0
    cur_dest.execute(insert, (row['id_cadorc'], row['num'], row['ano'], row['numorc'], row['dtorc'], row['descr'], row['prioridade'], row['obs'], row['status'], row['liberado'], row['codccusto'], row['liberado_tela'], empresa))
commit()

Erro ao mapear codccusto: 


### ICADORC

In [10]:
limpa_tabela(("icadorc",))

insert = cur_dest.prep("""
insert into icadorc (
    numorc,
    item,
    cadpro,
    qtd, 
    valor, 
    itemorc, 
    codccusto, 
    itemorc_ag,
    id_cadorc)
values(?,?,?,?,?,?,?,?,?)
""")

cur_orig.execute("""
select
	"NR_COTACAO" id_cadorc,
	a."NR_SEQ" item,
	"CD_PRODUTO" codreduz,
	"QTDE_ITENS" qtd
from
	"SCH"."ITENSCOT" a
join "SCH"."COTACOES" b using ("NR_COTACAO")
""")

info_cadorc = {k:[v1, v2] for k,v1,v2 in cur_dest.execute("select id_cadorc, numorc, codccusto from cadorc").fetchall()}

for row in cur_orig:
    try:
        cadpro = cadest[str(row['codreduz'])]
        numorc, codccusto = info_cadorc[row['id_cadorc']]
    except:
        print(f"Erro ao mapear codreduz: {row['codreduz']}")
        break
    cur_dest.execute(insert, (numorc, row['item'], cadpro, row['qtd'], 0, row['item'], codccusto, row['item'], row['id_cadorc']))
commit()

### FCADORC

In [11]:
limpa_tabela(("fcadorc",))

insert = cur_dest.prep("insert into fcadorc(numorc, codif, nome, valorc, id_cadorc) values (?,?,?,?,?)")

cur_orig.execute("""
with CTE as (
select
    concat(TO_CHAR(a."NR_COTACAO", 'fm00000/'),
           extract(year from b."DT_COTACAO")::int % 2000) as numorc,
    a."NR_ITEM" as ITEM,
    d."RECNUM" as codif,
    a."VL_UNITARIO" as vlruni,
    c."QTDE_ITENS" qtd,
    a."NR_COTACAO" as id_cadorc,
    d."RZ_SOCIAL" nome
from "SCH"."ARQ1591" a
join "SCH"."COTACOES" b 
    on a."NR_COTACAO" = b."NR_COTACAO"
left join "SCH"."ITENSCOT" c 
    on a."NR_COTACAO" = c."NR_COTACAO"
   and a."NR_ITEM"   = c."NR_ITEM"
   and a."CD_PRODUTO"= c."CD_PRODUTO"
left join "SCH"."FORNEC" d on a."CD_FORNEC" = d."NR_CGC_CPF"
order by 1,2,3)
select numorc, codif, nome, sum(qtd*vlruni) valorc, id_cadorc from cte group by 1,2,3,5
""")

for row in cur_orig:
	cur_dest.execute(insert, (
		row['numorc'],
		row['codif'],
		row['nome'][:70],
		row['valorc'],
		row['id_cadorc']
	))
commit()

### VCADORC

In [12]:
limpa_tabela(("vcadorc",))

insert = cur_dest.prep("""insert into vcadorc(numorc, item, codif, vlruni, vlrtot, ganhou, vlrganhou, classe, id_cadorc) values (?,?,?,?,?,?,?,?,?)""")

cur_orig.execute("""
with CTE as (
select
	concat(TO_CHAR(a."NR_COTACAO",
	'fm00000/'),
	extract(year
from
	b."DT_COTACAO")::int % 2000) as numorc,
	a."NR_ITEM" as ITEM,
	d."RECNUM" as codif,
	a."VL_UNITARIO" as vlruni,
	c."QTDE_ITENS" qtd,
	a."NR_COTACAO" as id_cadorc,
	d."RZ_SOCIAL" nome
from
	"SCH"."ARQ1591" a
join "SCH"."COTACOES" b 
    on
	a."NR_COTACAO" = b."NR_COTACAO"
left join "SCH"."ITENSCOT" c 
    on
	a."NR_COTACAO" = c."NR_COTACAO"
	and a."NR_ITEM" = c."NR_ITEM"
	and a."CD_PRODUTO" = c."CD_PRODUTO"
left join "SCH"."FORNEC" d on
	a."CD_FORNEC" = d."NR_CGC_CPF"
order by
	1,
	2,
	3)
select
	numorc, 
	item, 
	codif,
	vlruni,
	vlruni*qtd vlrtot,
	id_cadorc
from
	cte
""")

for row in cur_orig:
    cur_dest.execute(insert, (
        row['numorc'],
        row['item'],
        row['codif'],
        row['vlruni'],
        row['vlrtot'],
        None,
        None,
        'UN',
        row['id_cadorc']
    ))
commit()

cur_dest.execute("""merge into vcadorc a
using (
    SELECT 
        id_cadorc, 
        item, 
        codif, 
        vlruni, 
        ROW_NUMBER() OVER (
            PARTITION BY id_cadorc, item 
            ORDER BY vlruni ASC, codif ASC
        ) AS rn
    FROM vcadorc
) b
on a.id_cadorc = b.id_cadorc
and a.item = b.item
and b.rn = 1
when matched then
    update set 
        a.vlrganhou = b.vlruni,
        a.ganhou = b.codif""")
commit()

## LICITAÇÕES

In [13]:
#LIMPA LICITAÇÕES
cur_dest.execute("""
execute block as
    begin
    DELETE FROM regpreco;
    DELETE FROM regprecohis;
    DELETE FROM regprecodoc;
    DELETE FROM CADPROLIC_DETALHE_FIC;
    DELETE FROM CADPRO;
    DELETE FROM CADPRO_FINAL;
    DELETE FROM CADPRO_LANCE;
    DELETE FROM CADPRO_PROPOSTA;
    DELETE FROM PROLICS;
    DELETE FROM PROLIC;
    DELETE FROM CADPRO_STATUS;
    DELETE FROM CADLIC_SESSAO;
    DELETE FROM CADPROLIC_DETALHE;
    DELETE FROM CADPROLIC;
    DELETE FROM CADLIC;
    end;
""")
commit()

### CADLIC

In [14]:
# limpa_tabela(("cadlic",))

insert = cur_dest.prep("""
INSERT
		INTO
		cadlic(
		numlic,
		proclic,
		numero,
		ano,
		comp,
		licnova,
		liberacompra,
		discr,
		detalhe,
		registropreco,
		microempresa,
		numpro,
		discr7,
		datae,
		processo_data,
		dtadj,
		dthom,
		codtce,
		anomod,
		modlic,
		licit,
		codmod,
		dtpub,
		dtenc,
		valor,
		empresa,
		processo,
		processo_ano,
		dtreal,
        numorc,
        id_cadorc)
	VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
	?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
    ?, ?)
""")

cur_orig.execute("""
select
	"NR_COTACAO" numlic,
	to_char("NR_COTACAO", 'fm000000') num,
	substr(extract(year from "DT_COTACAO")::text,3,2) ano,
	coalesce("STATUS_LICITACAO",1) comp,
	1 licnova,
	case when "STATUS_LICITACAO" = 3 then 'S' else 'N' END liberacompra,
	substr("OBSERVACAO",1,1024) discr,
	coalesce("JUSTIFICATIVA", "OBSERVACAO") detalhe,
	case when "CD_MODALIDADE" in (11,14,24) then 'S' else 'N' end registropreco,
	'3' microempresa,
	"SEQ_PROCESSO" numpro,
	'Menor Preco Unitario' discr7,
	"DT_COTACAO" datae,
	coalesce("DATA_PROCESSO_ADMINISTRATIVO","DT_COTACAO") processo_data,
	"DT_ADJUDICACAO" dtadj,
	"DT_OMOLOGACAO" dthom,
	coalesce("ANO_PROCESSO"::int,extract(year from "DT_COTACAO")) anomod,
    "CD_MODALIDADE" codmod,
	coalesce("DATA_PUBLICACAO_EDITAL", "DT_PUB_EDITAL") dtpub,
	coalesce("DT_ENCERRA","DT_OMOLOGACAO") dtenc,
	"VL_PREVISTO" valor,
	"NR_COTACAO" processo,
	extract(year from "DT_COTACAO") processo_ano,
	"DT_COTACAO" dtreal,
	concat(TO_CHAR("NR_COTACAO", 'fm00000/'), extract(year from "DT_COTACAO")::int % 2000) as numorc,
	"NR_COTACAO" id_cadorc,
	"NR_PROCESSO" mascmod
from
	"SCH"."COTACOES"
where "DT_COTACAO" is not null
order by 1
""")

for row in cur_orig:
    info_modalidades = dict_modalidades.get(row['codmod'], {"Licit": "DISPENSA", "Modlic": "DI01", "Codmod": 1})

    licit = info_modalidades["Licit"]
    modlic = info_modalidades["Modlic"]
    codmod = info_modalidades["Codmod"]
    proclic = f'{row["num"]}/{row["ano"]}'

    cur_dest.execute(insert, (
		row['numlic'], 
		proclic,
		row['num'],
		f'20{row['ano']}',
		row['comp'],
		row['licnova'],
		row['liberacompra'],
		row['discr'],
		row['detalhe'],
		row['registropreco'],
		row['microempresa'],
		row['numpro'],
		row['discr7'],
		row['datae'],
		row['processo_data'],
		row['dthom'],
		row['dthom'],
		None,
		f'20{row['ano']}',
		modlic,
		licit,
		codmod,
		row['dtpub'],
		row['dthom'],
		row['valor'],
		empresa,
		row['processo'],
		row['processo_ano'],
		row['dtreal'],
		row['numorc'],
		row['id_cadorc']
	))
commit()

cur_dest.execute("""EXECUTE BLOCK AS
DECLARE VARIABLE DESCMOD VARCHAR(1024);
DECLARE VARIABLE CODMOD INTEGER;
BEGIN
    FOR
        SELECT CODMOD, DESCMOD FROM MODLIC INTO :CODMOD, :DESCMOD
    DO
    BEGIN
        UPDATE CADLIC SET LICIT = :DESCMOD where CODMOD = :CODMOD;
    END
END""")

cur_dest.execute("""UPDATE CADLIC SET FK_MODLICANO = (SELECT PK_MODLICANO FROM MODLICANO WHERE CODMOD = CADLIC.CODMOD AND ANOMOD = CADLIC.ANO AND CADLIC.EMPRESA = MODLICANO.EMPRESA) WHERE CODMOD IS NOT NULL""")
commit()

### CADPROLIC

In [15]:
cria_coluna("cadprolic", "nr_seq")
limpa_tabela(("cadlic_sessao", "cadprolic_detalhe_fic", "cadprolic_detalhe", "cadprolic"))

insert_cadprolic = cur_dest.prep("""
INSERT INTO
		cadprolic(item,
		item_mask,
		itemorc,
		cadpro,
		quan1,
		vamed1,
		vatomed1,
		codccusto,
		reduz,
		numlic,
		nr_seq)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select
	i."NR_SEQ" item,
	"CD_PRODUTO",
	"QTDE_ITENS" quan1,
	"VL_UNITARIO" vamed1,
	"VL_PREVISTO" vatomed1,
	0 codccusto,
	'N' reduz,
	"NR_COTACAO" numlic,
	"NR_ITEM"
	from
"SCH"."ITENSCOT" i
order by "NR_COTACAO", 1
""")

for row in cur_orig:
    try:
        cadpro = cadest[str(row['CD_PRODUTO'])]
    except KeyError:
        print(f"Erro ao mapear CD_PRODUTO: {row['CD_PRODUTO']}")
        quit()
    
    cur_dest.execute(insert_cadprolic, (
        row['item'],
        row['item'],
        row['item'],
        cadpro,
        row['quan1'],
        row['vamed1'],
        row['vatomed1'],
        row['codccusto'],
        row['reduz'],
        row['numlic'],
        row['NR_ITEM']
    ))
commit()

cur_dest.execute("""
	INSERT INTO CADPROLIC_DETALHE (NUMLIC,item,CADPRO,quan1,VAMED1,VATOMED1,marca,CODCCUSTO,ITEM_CADPROLIC, item_lc147)
	select numlic, item, cadpro, quan1, vamed1, vatomed1, marca, codccusto, item, item_lc147 from cadprolic b where
	not exists (select 1 from cadprolic_detalhe c where b.numlic = c.numlic and b.item = c.item);
""")
commit()

cur_dest.execute("""
	insert into 
	cadprolic_detalhe_fic (numlic, item, codigo, qtd, valor, qtdadt, valoradt, codccusto, qtdmed, valormed, tipo) 
	select numlic, item, item, quan1, vatomed1, quan1, vatomed1, codccusto, quan1, vatomed1, 'C' from cadprolic b where
	not exists (select 1 from cadprolic_detalhe_fic c where b.numlic = c.numlic and b.item = c.item);
""")
commit()

cur_dest.execute("""INSERT INTO CADLIC_SESSAO (NUMLIC, SESSAO, DTREAL, HORREAL, COMP, DTENC, HORENC, SESSAOPARA, MOTIVO) 
	SELECT L.NUMLIC, CAST(1 AS INTEGER), L.DTREAL, L.HORREAL, L.COMP, L.DTENC, L.HORENC, CAST('T' AS VARCHAR(1)), CAST('O' AS VARCHAR(1)) FROM CADLIC L 
	WHERE numlic not in (SELECT FIRST 1 S.NUMLIC FROM CADLIC_SESSAO S WHERE S.NUMLIC = L.NUMLIC)""")
commit()

### PROLIC & PROLICS

In [16]:
limpa_tabela(("prolics","prolic"))

insert_prolic = cur_dest.prep("insert into prolic(numlic,codif,nome,status) values(?,?,?,?)")
insert_prolics = cur_dest.prep("insert into prolics(sessao,numlic,codif,habilitado,status,cpf,representante) values(?,?,?,?,?,?,?)")

cur_orig.execute("""
select DISTINCT
	"NR_COTACAO" numlic,
	b."RECNUM" codif,
	SUBSTR(b."RZ_SOCIAL",1,40) s_fornecedor,
	'C' status,
	a."CD_FORNEC" cnpj
from
	"SCH"."PRECOS" a
join "SCH"."FORNEC" b on a."CD_FORNEC" = b."NR_CGC_CPF"
""")

for row in cur_orig:
    cur_dest.execute(insert_prolic, (row['numlic'], row['codif'], to_cp1252_safe(row['s_fornecedor']), 'A'))
    cur_dest.execute(insert_prolics, (1, row['numlic'], row['codif'], 'S', 'A', row['cnpj'], to_cp1252_safe(row['s_fornecedor'])))
commit()

### PROPOSTA

In [ ]:
limpa_tabela(("cadpro", "cadpro_final", "cadpro_proposta"))

insert = cur_dest.prep("""
INSERT
    INTO
    cadpro_proposta(sessao,
    codif,
    item,
    itemp,
    quan1,
    vaun1,
    vato1,
    numlic,
    status,
    subem,
    marca,
    itemlance)
VALUES(1,?,?,?,?,?,?,?,?,?,?,'S')""")

insert_lances = cur_dest.prep("""
insert into cadpro_lance (sessao, rodada, codif, itemp, vaunl, vatol, status, subem, numlic)
values (1,1,?,?,?,?,?,?,?,?)
""")

insert_final = cur_dest.prep("""
insert into cadpro_final (numlic, ult_sessao, codif, itemp, vaunf, vatof, STATUS, subem)
values (?,1,?,?,?,?,?,?)
""")

cur_orig.execute("""
select distinct
	b."RECNUM" codif,
	a."NR_SEQ" item,
	a."QT_COTADA" quan1,
	a."VL_UNITARIO" vaun1,
	a."VL_TOTAL" vato1,
	a."NR_COTACAO" numlic,
  	case when coalesce(a."ITEM_DESCLAS", '0') = '1' then 'D' else 'C' end status_proposta,
  	case when coalesce(a."HOMOLOGA", '0') = '1' and coalesce(a."ITEM_DESCLAS", '0') = '0' then 'F' else 'C' end status_homologacao,
  	"HOMOLOGA" subem,
  	substr(a."DS_MARCA",1,50) marca,
  	'S' itemlance
from
	"SCH"."PRECOS" a
join "SCH"."FORNEC" b on a."CD_FORNEC" = b."NR_CGC_CPF"
join "SCH"."HOMOLOGACAO_ITEM" c using ("NR_COTACAO", "NR_SEQ", "NR_CGC_CPF")
""")

for row in cur_orig:   
    cur_dest.execute(insert, (
        row['codif'],
        row['item'],
        row['item'],
        row['quan1'],
        row['vaun1'],
        row['vato1'],
        row['numlic'],
        row['status_proposta'],
        row['subem'],
        to_cp1252_safe(row['marca'])
    ))

    cur_dest.execute(insert_lances, (
        row['codif'],
        row['item'],
        row['vaun1'],
        row['vato1'],
        row['status_homologacao'],
        row['subem'],
        row['numlic']
    ))

    cur_dest.execute(insert_final, (
        row['numlic'],
        row['codif'],
        row['item'],
        row['vaun1'],
        row['vato1'],
        row['status_homologacao'],
        row['subem']
    ))
commit()

cur_dest.execute("""update cadlic a SET comp = 3 WHERE comp = 1 AND EXISTS (SELECT 1 FROM cadpro_final x WHERE subem = 1 AND x.numlic = a.numlic)""")
commit()

### LANCES

In [18]:
limpa_tabela(("cadpro_lance",))
cur_dest.execute("""insert into cadpro_lance (sessao, rodada, codif, itemp, vaunl, vatol, status, subem, numlic)
	SELECT sessao, 1 rodada, CODIF, ITEMP, VAUN1, VATO1, 'F' status, SUBEM, numlic FROM CADPRO_PROPOSTA cp where subem = 1 and not exists
	(select 1 from cadpro_lance cl where cp.codif = cl.codif and cl.itemp = cp.itemp and cl.numlic = cp.numlic)""")
commit()

### FINAL

In [19]:
cur_dest.execute("""INSERT into cadpro_final (numlic, ult_sessao, codif, itemp, vaunf, vatof, STATUS, subem)
	SELECT numlic, sessao, codif, itemp, vaun1, vato1, CASE WHEN status = 'F' THEN 'C' ELSE status end, subem FROM cadpro_proposta
	WHERE NOT EXISTS (SELECT 1 FROM cadpro_final f WHERE f.numlic = cadpro_proposta.numlic AND f.itemp = cadpro_proposta.itemp AND f.codif = cadpro_proposta.codif)""")
commit()

cur_dest.execute("""update cadlic a SET comp = 3 WHERE comp = 1 AND EXISTS (SELECT 1 FROM cadpro_final x WHERE subem = 1 AND x.numlic = a.numlic)""")
commit()

### STATUS

In [20]:
cur_dest.execute("""
    INSERT INTO cadpro_status (numlic, sessao, itemp, item, telafinal)
    SELECT b.NUMLIC, 1 AS sessao, a.item, a.item, 'I_ENCERRAMENTO'
    FROM CADPROLIC a
    JOIN cadlic b ON a.NUMLIC = b.NUMLIC
    WHERE NOT EXISTS (
        SELECT 1
        FROM cadpro_status c
        WHERE a.numlic = c.numlic
    )
    AND EXISTS (
        SELECT 1
        FROM cadlic d
        WHERE d.numlic = a.numlic AND d.comp = 3
    );
""")
commit()

### CADPRO

In [21]:
limpa_tabela(("cadpro",))

cur_dest.execute("""
INSERT INTO CADPRO (
  CODIF, CADPRO, QUAN1, VAUN1, VATO1, SUBEM, STATUS,
  ITEM, NUMORC, ITEMORCPED, CODCCUSTO, FICHA, ELEMENTO, DESDOBRO,
  NUMLIC, ULT_SESSAO, ITEMP, QTDADT, QTDPED, VAUNADT, VATOADT,
  PERC, QTDSOL, ID_CADORC, VATOPED, VATOSOL, TPCONTROLE_SALDO,
  QTDPED_FORNECEDOR_ANT, VATOPED_FORNECEDOR_ANT, MARCA
)
WITH dados AS (
  SELECT
    a.CODIF,
      c.CADPRO,
      c.ITEM,
      d.NUMORC,
      c.CODCCUSTO,
      c.FICHA,
      c.ELEMENTO,
      c.DESDOBRO,
      a.NUMLIC,
      b.ITEMP,
      d.ID_CADORC,
      a.marca,
      a.quan1  AS qtdunit,
      a.VAUN1 AS vaun,
      a.vato1 AS vatotal
  FROM CADPRO_PROPOSTA a
  JOIN CADPRO_STATUS b
    ON b.NUMLIC = a.NUMLIC
    AND b.ITEMP  = a.ITEMP
    AND b.SESSAO = a.SESSAO
  JOIN CADPROLIC_DETALHE c
    ON c.NUMLIC = a.NUMLIC
    AND b.ITEMP  = c.ITEM_CADPROLIC
  JOIN CADLIC d
    ON d.NUMLIC = a.NUMLIC
  WHERE a.SUBEM = 1
    AND a.STATUS = 'C'
    AND d.ANO >= 19
)
SELECT
    CODIF,
    CADPRO,
    qtdunit,
    vaun,
    vatotal,
    1,
    'C',
    ITEM,
    NUMORC,
    ITEM,
    CODCCUSTO,
    FICHA,
    ELEMENTO,
    DESDOBRO,
    NUMLIC,
    1,
    ITEMP,
    qtdunit,
    0,
    vaun,
    vatotal,
    0,
    0,
    ID_CADORC,
    0,
    0,
    'Q',
    0,
    0,
    marca
FROM dados d
WHERE NOT EXISTS (
    SELECT 1
    FROM CADPRO cp
    WHERE cp.NUMLIC = d.NUMLIC
      AND cp.ITEM   = d.ITEM
      AND cp.CODIF  = d.CODIF
)""")
commit()

### REGPRECO

In [22]:
cur_dest.execute("""
EXECUTE BLOCK AS  
    BEGIN  
    INSERT INTO REGPRECODOC (NUMLIC, CODATUALIZACAO, DTPRAZO, ULTIMA)  
    SELECT DISTINCT A.NUMLIC, 0, DATEADD(1 YEAR TO A.DTHOM), 'S'  
    FROM CADLIC A WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECODOC X  
    WHERE X.NUMLIC = A.NUMLIC);  

    INSERT INTO REGPRECO (COD, DTPRAZO, NUMLIC, CODIF, CADPRO, CODCCUSTO, ITEM, CODATUALIZACAO, QUAN1, VAUN1, VATO1, QTDENT, SUBEM, STATUS, ULTIMA)  
    SELECT B.ITEM, DATEADD(1 YEAR TO A.DTHOM), B.NUMLIC, B.CODIF, B.CADPRO, B.CODCCUSTO, B.ITEM, 0, B.QUAN1, B.VAUN1, B.VATO1, 0, B.SUBEM, B.STATUS, 'S'  
    FROM CADLIC A INNER JOIN CADPRO B ON (A.NUMLIC = B.NUMLIC) WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECO X  
    WHERE X.NUMLIC = B.NUMLIC AND X.CODIF = B.CODIF AND X.CADPRO = B.CADPRO AND X.CODCCUSTO = B.CODCCUSTO AND X.ITEM = B.ITEM);  

    INSERT INTO REGPRECOHIS (NUMLIC, CODIF, CADPRO, CODCCUSTO, ITEM, CODATUALIZACAO, QUAN1, VAUN1, VATO1, SUBEM, STATUS, MOTIVO, MARCA, NUMORC, ULTIMA)  
    SELECT B.NUMLIC, B.CODIF, B.CADPRO, B.CODCCUSTO, B.ITEM, 0, B.QUAN1, B.VAUN1, B.VATO1, B.SUBEM, B.STATUS, B.MOTIVO, B.MARCA, B.NUMORC, 'S'  
    FROM CADLIC A INNER JOIN CADPRO B ON (A.NUMLIC = B.NUMLIC) WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECOHIS X  
    WHERE X.NUMLIC = B.NUMLIC AND X.CODIF = B.CODIF AND X.CADPRO = B.CADPRO AND X.CODCCUSTO = B.CODCCUSTO AND X.ITEM = B.ITEM);  
END;""")
commit()

# PEDIDOS

In [28]:
limpa_tabela(("icadped", "cadped"))

inser_cadped = cur_dest.prep("insert into cadped (numped, num, ano, datped, codif, id_cadped, empresa, numlic, obs, codccusto) values (?,?,?,?,?,?,?,?,?,?)")
insert_icadped = cur_dest.prep("insert into icadped (numped, item, cadpro, qtd, prcunt, prctot, codccusto, id_cadped) values (?,?,?,?,?,?,?,?)")

cur_orig.execute("""
with cte as
(select
	concat(to_char("NR_PEDIDO", 'fm00000/'), extract(year from "DT_PEDIDO")%2000) numped,
	to_char("NR_PEDIDO", 'fm00000') num,
	extract(year from "DT_PEDIDO") ano,
	"DT_PEDIDO" dataped,
	b."RECNUM" codif,
	a."NR_PEDIDO" id_cadped,
	a."NR_COTACAO" numlic,
	a."DS_OBS" obs,
	c."RECNUM" codccusto,
	item,
	codreduz,
	qtd,
	prcunt,
	prctot
from
	"SCH"."CAPAPED" a
left join "SCH"."FORNEC" b using ("NR_CGC_CPF")
left join "SCH"."CADSET" c using ("ID_CADSET")
left join (select
	"NR_SEQ" item,
	"CD_PRODUTO" codreduz,
	"QT_PEDIDA"-"QT_ANULADA" qtd,
	"VL_UNITARIO" prcunt,
	("QT_PEDIDA"-"QT_ANULADA")*"VL_UNITARIO" prctot,
	"NR_PEDIDO"
from
	"SCH"."ITEMPED") d using ("NR_PEDIDO"))
select * from CTE where ano = 2025
order by id_cadped
""")

id_cadped = 0

for row in cur_orig:
    if row['id_cadped'] != id_cadped:
        id_cadped = row['id_cadped']
        cur_dest.execute(inser_cadped, (
            row['numped'],
            row['num'],
            row['ano'],
            row['dataped'],
            row['codif'],
            id_cadped,
            empresa,
            row['numlic'],
            row['obs'],
            row['codccusto']
        ))
        commit()
    
    try:
        cadpro = cadest[str(row['codreduz'])]
    except:
        print(f'erro ao encontrar cadpro para o item {row["item"]}')
        break

    cur_dest.execute(insert_icadped, (
        row['numped'],
        row['item'],
        cadpro,
        row['qtd'],
        row['prcunt'],
        row['prctot'],
        row['codccusto'],
        id_cadped
    ))
commit()

cur_dest.execute("""
UPDATE cadped a SET a.codatualizacao_rp = (SELECT max(codatualizacao) 
FROM regprecodoc x WHERE x.numlic = a.numlic)
""")
commit()

### SUBPEDIDOS

In [32]:
limpa_tabela(("icadped a where exists (select 1 from cadped x where id_cadpedlicit is not null and x.id_cadped = a.id_cadped)", "cadped where id_cadpedlicit is not null"))
pedidos_pai = {k:{'numped': v0, 'codif': v1, 'codccusto': v2, 'numlic': v3} for k, v0, v1, v2, v3 in cur_dest.execute("select id_cadped, numped, codif, codccusto, numlic from cadped where id_cadpedlicit is null").fetchall()}

inser_cadped = cur_dest.prep("insert into cadped (numped, num, ano, datped, codif, id_cadped, empresa, numlic, obs, codccusto, id_cadpedlicit) values (?,?,?,?,?,?,?,?,?,?,?)")
insert_icadped = cur_dest.prep("insert into icadped (numped, item, cadpro, qtd, prcunt, prctot, codccusto, id_cadped) values (?,?,?,?,?,?,?,?)")

cur_orig.execute("""
select
	concat(to_char(a."NR_ENTREGA", 'fm00000/'), extract(year from "DT_ENTREGA")%2000) numped,
	to_char(a."NR_ENTREGA", 'fm00000') num,
	extract(year from "DT_ENTREGA") ano,
	"DT_ENTREGA" datped,
	a."NR_PEDIDO" id_cadpedlicit,
	"OBSERVACAO" obs,
	b."NR_SEQ" item,
	b."CD_PRODUTO" codreduz,
	b."QTD_PEDIDA" qtd,
	b."VL_UNITARIO" prcunt,
	"VL_TOTAL" prctot
from
	"SCH"."ARQ1223" a
join "SCH"."ARQ1231" b on
	b."NR_ENTREGA" = a."NR_ENTREGA"
where
	extract(year from "DT_ENTREGA") = 2025
	and exists (
	select
		1
	from
		"SCH"."CAPAPED" x
	where
		x."NR_PEDIDO" = a."NR_PEDIDO"
		and extract(year from x."DT_PEDIDO") = 2025)
order by
	2,
	b."NR_SEQ"
""")

id = cur_dest.execute("select max(id_cadped) from cadped").fetchone()[0]
subped_ant = 0
pedido_pai = 0

for row in cur_orig:
    if row['numped'] != subped_ant:
        id += 1
        subped_ant = row['numped']
        pedido_pai = pedidos_pai[row['id_cadpedlicit']]

        cur_dest.execute(inser_cadped, (
            row['numped'],
            row['num'],
            row['ano'],
            row['datped'],
            pedido_pai['codif'],
            id,
            empresa,
            pedido_pai['numlic'],
            row['obs'],
            pedido_pai['codccusto'],
            row['id_cadpedlicit']
        ))
        commit()
        
    cadpro = cadest[str(row['codreduz'])]
    cur_dest.execute(insert_icadped, (
        row['numped'],
        row['item'],
        cadpro,
        row['qtd'],
        row['prcunt'],
        row['prctot'],
        pedido_pai['codccusto'],
        id
    ))
commit()

### EXERCICIOS ANTERIORES

In [33]:
limpa_tabela(("cadpro_saldo_ant", "regpreco_saldo_ant"))

insert = cur_dest.prep("""
insert into cadpro_saldo_ant (
    ano,
    numlic,
    item,
    cadpro,
    qtdped,
    vatoped)
values (?,?,?,?,?,?)
""")

cur_orig.execute("""
with cte as
(select
	concat(to_char("NR_PEDIDO", 'fm00000/'), extract(year from "DT_PEDIDO")%2000) numped,
	to_char("NR_PEDIDO", 'fm00000') num,
	extract(year from "DT_PEDIDO") ano,
	"DT_PEDIDO" dataped,
	b."RECNUM" codif,
	a."RECNUM" id_cadped,
	a."NR_COTACAO" numlic,
	a."DS_OBS" obs,
	c."RECNUM" codccusto,
	item,
	codreduz,
	qtd,
	prcunt,
	prctot
from
	"SCH"."CAPAPED" a
left join "SCH"."FORNEC" b using ("NR_CGC_CPF")
left join "SCH"."CADSET" c using ("ID_CADSET")
left join (select
	"NR_SEQ" item,
	"CD_PRODUTO" codreduz,
	"QT_PEDIDA"-"QT_ANULADA" qtd,
	"VL_UNITARIO" prcunt,
	("QT_PEDIDA"-"QT_ANULADA")*"VL_UNITARIO" prctot,
	"NR_PEDIDO"
from
	"SCH"."ITEMPED") d using ("NR_PEDIDO"))
select numlic, codreduz, item, sum(qtd) qtdped, sum(prctot) vatoped from CTE where ano < 2025
and item is not null
group by 1,2,3
""")

for row in cur_orig:     
    cadpro = cadest[str(row['codreduz'])]
    try:
        cur_dest.execute(insert, (
			int(exercicio)-1,
			row['numlic'],
			row['item'],
			cadpro,
			row['qtdped'],
			row['vatoped']
		))
        commit()
    except fdb.DatabaseError as e:
        if e.args[1] == -803: 
            cur_dest.execute("update cadpro_saldo_ant set qtdped = qtdped + ?, vatoped = vatoped + ? where numlic = ? and item = ?", (row['qtdped'], row['vatoped'], row['numlic'], row['item'])) 
            commit()
        else:
            raise

# ESTOQUE

## SALDO INICIAL

In [35]:
limpa_tabela(("icadreq where id_requi = 0", "requi where id_requi = 0"))
cur_dest.execute("alter trigger TAU_ESTOQUE active")
commit()

cur_dest.execute(f"""
INSERT
    INTO
    requi (empresa,
    id_requi,
    requi,
    num,
    ano,
    destino,
    codccusto,
    datae,
    dtlan,
    dtpag,
    entr,
    said,
    comp,
    codif,
    entr_said)
VALUES (
    {empresa},
    0,
    '000000/{exercicio[-2:]}',
    '000000',
    {exercicio},
    '000000000',
    0,
    '31.12.{int(exercicio)-1}',
    '31.12.{int(exercicio)-1}',
    null,
    'S',
    'N',
    'P',
    0,
    null)
""")

insert_icadreq = cur_dest.prep("""
    insert into icadreq (id_requi, requi, codccusto, empresa, item, quan1, quan2, vaun1, vaun2, vato1, vato2, cadpro, destino) values (?,?,?,?,?,?,?,?,?,?,?,?,?)
""")

cur_orig.execute(f"""
with CTE as (
select
	0 id_requi,
	'000000/{exercicio[-2:]}' requi,
	b."RECNUM" codccusto,
	coalesce(k."CD_PRODUTO", '0') codreduz,
	k."QT_SALDO" quan1,
	k."VL_PRECO_MEDIO" vaun1,
	k."QT_SALDO"*k."VL_PRECO_MEDIO" vato1,
	to_char("CD_LOCAL", 'fm000000000') destino,
	row_number() over (partition by "CD_LOCAL", "CD_PRODUTO" order by k."RECNUM" desc) rn
from
	"SCH"."KARDEX" k
left join "SCH"."CADSET" b using ("ID_CADSET")
join "SCH"."PRODUTOS" c using ("CD_PRODUTO")
where "NR_ANO" < {exercicio})
select * from cte where rn = 1
""")

item = 0

for row in cur_orig:
    item += 1
    try:
        cadpro = cadest[str(row['codreduz'])]
    except:
        print(f'Erro ao obter cadpro para o item {row["codreduz"]}. Verifique se o item está cadastrado.')
        break

    cur_dest.execute(insert_icadreq, (row['id_requi'], row['requi'], row['codccusto'], empresa, item, row['quan1'], 0, row['vaun1'], 0, row['vato1'], 0, cadpro, row['destino']))
commit()

cur_dest.execute("UPDATE icadreq SET fK_destino = (SELECT pk_destino FROM destino x WHERE x.cod = destino)")
commit()

### REQUISIÇÕES

In [ ]:
limpa_tabela(('icadreq where id_requi <> 0', 'requi where id_requi <> 0'))

insert_requi = cur_dest.prep("""
INSERT
    INTO
    requi (empresa,
    id_requi,
    requi,
    num,
    ano,
    destino,
    codccusto,
    datae,
    dtlan,
    dtpag,
    entr,
    said,
    comp,
    codif,
    entr_said,
    docum,
    obs)
VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
""")

cur_orig.execute(f"""
select
    k."NR_DOC" id_requi,
	concat(format("NR_DOC", '000000'), extract(year from "DT_MOVIMENTO")::int%2000) requi,
	format("NR_DOC", '000000') num,
	extract(year from "DT_MOVIMENTO") ano,
	to_char("CD_LOCAL",'fm000000000') destino,
	setor."RECNUM" codccusto,
	"DT_MOVIMENTO" data,
	case when ("QT_ENTRADAS" <> 0) and ("QT_SAIDAS" = 0) then 'E'
	when ("QT_ENTRADAS" = 0) and ("QT_SAIDAS" <> 0) then 'S'
	else 'X' end tipo,
    "TP_DOC" obs,
	"NR_DOC" docum,
	row_number() over (partition by "NR_DOC" order by "CD_PRODUTO") item,
	"CD_PRODUTO" codreduz,
	"QT_ENTRADAS" quan1,
	"VL_UNITARIO" vaun1,
	"QT_ENTRADAS"*"VL_UNITARIO" vato1,
	"QT_SAIDAS" quan2,
	case when "QT_SAIDAS" = 0 then 0 else "VL_UNITARIO" end vaun2,
	"VL_UNITARIO"*"QT_SAIDAS" vato2
from
	"SCH"."KARDEX" k
join "SCH"."CADSET" setor using ("ID_CADSET")
where extract(year from "DT_MOVIMENTO") = {exercicio}
order by 2
""")

requi_ant = 0

for row in cur_orig:
    if row['id_requi'] != requi_ant:
        requi_ant = row['id_requi']
        cur_dest.execute(insert_requi, (
            empresa,
            row['id_requi'],
            row['requi'],
            row['num'],
            row['ano'],
            row['destino'],
            row['codccusto'],
            None,
            row['data'],
            None,
            None,
            None,
            'P',
            0,
            None,
            row['docum'],
            row['obs']
        ))
        commit()

    cadpro = cadest[str(row['codreduz'])]
    cur_dest.execute(insert_icadreq, (
        row['id_requi'],
        row['requi'],
        row['codccusto'],
        empresa,
        row['item'],
        row['quan1'],
        row['quan2'],
        row['vaun1'],
        row['vaun2'],
        row['vato1'],
        row['vato2'],
        cadpro,
        row['destino']
    ))
commit()